## Example of reading our "annotated data"

In [1]:
import json
import copy

import numpy as np
import pandas as pd

In [9]:
data = pd.read_excel("annotation-2021Apr16-total-public_June01-filtered.xlsx", index_col='q_id').drop("Annotator", axis=1)

In [10]:
data.head(3)

,question_value,new_question,answer_value,new_answer,drop,reason,snippet
q_id,,,,,,,
530a5117970c65fa6b000007,"What is the meaning of the acronym ""TAILS"" use...",NaN,['TAILS: Terminal Amine Isotopic Labeling of S...,['Terminal Amine Isotopic Labeling of Substrat...,NaN,NaN,## sniidx : 0 ## : . It is important to identi...
54ede8c594afd61504000009,In which phase of the cell cycle arrest is imp...,NaN,"['In Fanconi anemia cells, the S-phase checkpo...","['the S-phase', 'S-phase']",NaN,NaN,## sniidx : 0 ## : We found that ICLs activate...
56c868a95795f9a73e000017,What is the application of the Bimolecular Flu...,NaN,['The study of protein-protein interactions in...,"['the study of protein interactions', 'study o...",NaN,NaN,## sniidx : 0 ## : Bimolecular fluorescence co...


In [15]:
# Example of Train data
train=dict()

# Please download training data from the official BioASQ website : http://participants-area.bioasq.org/datasets/
with open("2021-challenge-data/BioASQ-original/BioASQ-9b/train-raw/BioASQ-trainingDataset9b.json", "r") as reader:
    train["9b"]=json.load(reader)['questions']
    
train["9b"][10]

{'body': 'Name synonym of Acrokeratosis paraneoplastica.',
 'documents': ['http://www.ncbi.nlm.nih.gov/pubmed/3819049',
  'http://www.ncbi.nlm.nih.gov/pubmed/18775590',
  'http://www.ncbi.nlm.nih.gov/pubmed/6225397',
  'http://www.ncbi.nlm.nih.gov/pubmed/22146896',
  'http://www.ncbi.nlm.nih.gov/pubmed/7796616',
  'http://www.ncbi.nlm.nih.gov/pubmed/18672707',
  'http://www.ncbi.nlm.nih.gov/pubmed/16435144',
  'http://www.ncbi.nlm.nih.gov/pubmed/7640201',
  'http://www.ncbi.nlm.nih.gov/pubmed/22252191',
  'http://www.ncbi.nlm.nih.gov/pubmed/17097409',
  'http://www.ncbi.nlm.nih.gov/pubmed/17374318',
  'http://www.ncbi.nlm.nih.gov/pubmed/22470801',
  'http://www.ncbi.nlm.nih.gov/pubmed/8949310',
  'http://www.ncbi.nlm.nih.gov/pubmed/1521479',
  'http://www.ncbi.nlm.nih.gov/pubmed/1433123'],
 'ideal_answer': ['Acrokeratosis paraneoplastic (Bazex syndrome) is a rare, but distinctive paraneoplastic dermatosis characterized by erythematosquamous lesions located at the acral sites and is mos

### Modifying

In [17]:
#checking code
test=False
dropCnt = 0

outData = copy.deepcopy(train["9b"])
print("outData: ", len(outData))
    
# question iter loop
print("\n" + "#####"*5 + " Processing question modification " + "#####"*5 + "\n")  
for iteridx, paragraph in enumerate(train["9b"]):
    if paragraph[u'type']!='factoid':
        continue
    if paragraph[u'id'] not in data["new_question"]:
        continue
        
    if type(data["new_question"][paragraph[u'id']]) == float and np.isnan(data["new_question"][paragraph[u'id']]):
        pass
    elif type(data["new_question"][paragraph[u'id']]) == str:
        print("for id %s, Question corrected! \n-> Was : %s"%(paragraph[u'id'], outData[iteridx]['body']))
        wasQbody = outData[iteridx]['body']
        outData[iteridx]['body'] = data["new_question"][paragraph[u'id']]
        print("-> Now : %s"%outData[iteridx]['body'])
    else:
        print("Fatal error. Abort!")
        raise
        
print("\n" + "#####"*5 + " Processing answer modification " + "#####"*5 + "\n")            

for iteridx, paragraph in enumerate(train["9b"]):
    if paragraph[u'type']!='factoid':
        continue
    if paragraph[u'id'] not in data["new_question"]:
        continue
    
    new_answer = data["new_answer"][paragraph[u'id']]
    
    if type(new_answer) == float and np.isnan(new_answer):
        pass
    elif type(new_answer) == str:
        try:
            new_answer_list = eval(new_answer)
            if type(new_answer_list) == list:
                print("for id %s, new answer given!"%paragraph[u'id'])
                print("-> Was : %s"%outData[iteridx]['exact_answer'])
                outData[iteridx]['exact_answer'] = new_answer_list
                print("-> Now : %s"%new_answer) 
            else:
                print("@@ ## eval(new_answer) works but it is not a list!", paragraph[u'id'], new_answer)
                break
        except:
            print("@@ new_answer : %s However, it seems like it is not a new answer suggestion."%new_answer)
            break
        
    if data["drop"][paragraph[u'id']] ==1.0:
        if type(new_answer) == float and np.isnan(new_answer):
            print("for id %s, drop question! Current Answer : %s"%(paragraph[u'id'],data["answer_value"][paragraph[u'id']]))
            #outData[iteridx]['exact_answer'] = ["DROP_THE_QUESTION_NO_MATCH"]
            outData[iteridx]=""
            dropCnt += 1
        else:
            print("for id %s, drop selected but new answer given!"%paragraph[u'id'])
            print("Current Answer : %s"%data["answer_value"][paragraph[u'id']])
            print("New Answer : %s"%new_answer)
    else:
        pass

print("outData: ", len(outData))
print("dropCnt: ",dropCnt)

outData:  3743

######################### Processing question modification #########################

for id 532366f09b2d7acc7e000015, Question corrected! 
-> Was : Which amino acid residue appears mutated in most of the cases reported with  cadasil syndrome?
-> Now : Which amino acid residue appears mutated in most of the cases reported with cadasil syndrome?
for id 52f350042059c6d71c000010, Question corrected! 
-> Was : What is the incidence of Edwards syndrom in the european population?
-> Now : What is the incidence of Edwards syndrome in the european population?
for id 53189656b166e2b80600001c, Question corrected! 
-> Was : Against which protein is the antibody used for immonostaining of Lewy bodies raised?
-> Now : Against which protein is the antibody used for immunostaining of Lewy bodies raised?
for id 5540fbce234c5a7c75000001, Question corrected! 
-> Was : Which autophagy pathway is trigered by the KFERQ motif of cytosolic proteins?
-> Now : Which autophagy pathway is trigger

In [18]:
outData9b = list(filter(None, outData))
print("outData: ", len(outData9b))

outData:  3719


In [19]:
data.head(3)

,question_value,new_question,answer_value,new_answer,drop,reason,snippet
q_id,,,,,,,
530a5117970c65fa6b000007,"What is the meaning of the acronym ""TAILS"" use...",NaN,['TAILS: Terminal Amine Isotopic Labeling of S...,['Terminal Amine Isotopic Labeling of Substrat...,NaN,NaN,## sniidx : 0 ## : . It is important to identi...
54ede8c594afd61504000009,In which phase of the cell cycle arrest is imp...,NaN,"['In Fanconi anemia cells, the S-phase checkpo...","['the S-phase', 'S-phase']",NaN,NaN,## sniidx : 0 ## : We found that ICLs activate...
56c868a95795f9a73e000017,What is the application of the Bimolecular Flu...,NaN,['The study of protein-protein interactions in...,"['the study of protein interactions', 'study o...",NaN,NaN,## sniidx : 0 ## : Bimolecular fluorescence co...


In [20]:
# questionCheck = 
sum([1 for ele in data['new_question'].fillna(0) if ele != 0])

22

## Save

In [29]:
with open("BioASQ-trainingDatase9b-annot-2021Apr21.json", "w") as reader:
    json.dump(fp=reader, obj={"questions":outData9b}, indent=2)